In [1]:
import collections
import os
import itertools
import time

import datasets
import rich
import torch
from tqdm import tqdm
import transformers
import trlx
from trlx.data.configs import TRLConfig

/home/mila/g/gagnonju/.main_anaconda/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
ds_train = datasets.load_dataset("gigaword", split="train")
# ds_eval  = datasets.load_dataset("gigaword", split="validation")

Found cached dataset gigaword (/home/mila/g/gagnonju/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6)


In [3]:
class LMDataset(torch.utils.data.Dataset):
    def __init__(self, ds):
        self.ds = ds
    
    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        return self.ds[idx]["document"]

ds_train_obj = torch.utils.data.Subset(LMDataset(ds_train), range(10000))
# ds_eval_obj  = torch.utils.data.Subset(LMDataset(ds_eval),  range(1000))
reward_tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")

stuff = collections.Counter()
for entry in ds_train_obj:
    input_ids = reward_tokenizer(entry)["input_ids"]
    stuff.update([len(input_ids)])
    
rich.print(max(stuff.keys()))

83

In [4]:
reward_model_model_name = "gpt2"
reward_tokenizer = transformers.AutoTokenizer       .from_pretrained(reward_model_model_name)
reward_tokenizer.pad_token = reward_tokenizer.eos_token
reward_model     = transformers.AutoModelForCausalLM.from_pretrained(reward_model_model_name).cuda()

for param in reward_model.parameters():
    param.requires_grad = False

In [5]:
def rl_distillation(samples):
    with torch.no_grad():
        tokenized = reward_tokenizer(
            samples, 
            padding        = True, 
            truncation     = True,
            return_tensors = "pt", 
        )
        tokenized = {k: v.cuda() for k, v in tokenized.items()}
        
        # Good
        logp = reward_model(**tokenized).logits.log_softmax(-1)
        
        # Likely good
        logp = logp.gather(
            dim=-1, 
            index=tokenized["input_ids"].unsqueeze(-1)
        ).squeeze(-1)
        
        assert logp.shape == (len(samples), logp.shape[1]), logp.shape
        per_batch_logp = logp.sum(-1)
        
        assert per_batch_logp.shape == (len(samples),), per_batch_logp.shape

        return per_batch_logp
    
CONFIG_PATH = "/home/mila/g/gagnonju/Marg-Li-CoT/our_scratchpad/configs/ppo_config.yml"
model = trlx.train(
    "distilgpt2", 
    config=TRLConfig.load_yaml(CONFIG_PATH),
    prompts      = ds_train_obj,
    eval_prompts = ds_train_obj,
    reward_fn    = rl_distillation,
)

`trlx.train()`

`reward_fn is not None`

config = TRLConfig(model=ModelConfig(model_path='distilgpt2', tokenizer_path='gpt2', 
model_type='AcceleratePPOModel', num_layers_unfrozen=2), train=TrainConfig(total_steps=10000, seq_length=100, 
epochs=1000, batch_size=128, lr_ramp_steps=100, lr_decay_steps=79000, weight_decay=1e-06, 
learning_rate_init=0.0001412, learning_rate_target=0.0001412, opt_betas=[0.9, 0.95], checkpoint_interval=10000, 
eval_interval=16, pipeline='PPOPipeline', orchestrator='PPOOrchestrator', checkpoint_dir='ckpts', 
project_name='trlx', entity_name=None, seed=1000), method=PPOConfig(name='ppoconfig', ppo_epochs=4, 
num_rollouts=128, chunk_size=128, init_kl_coef=0.2, target=6, horizon=10000, gamma=1, lam=0.95, cliprange=0.2, 
cliprange_value=0.2, vf_coef=2.3, gen_kwargs={'max_new_tokens': 1, 'min_length': 0, 'top_k': 0.0, 'top_p': 1.0, 
'do_sample': True}))

`get_model`

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

/home/mila/g/gagnonju/Marg-Li-CoT/our_scratchpad/trlx_repo/trlx/model/accelerate_base_model.py:211: TqdmExperimentalWarning: rich is experimental/alpha
  tbar = tqdm(


EPOCH 1 #####

EPOCH 2 #####

EPOCH 3 #####

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2172386/3950854667.py:28 in <cell line: 28>                                       │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2172386/3950854667.py'                      │
│                                                                                                  │
│ /home/mila/g/gagnonju/Marg-Li-CoT/our_scratchpad/trlx_repo/trlx/trlx.py:122 in train             │
│                                                                                                  │
│   119 │   │   raise ValueError(f"Either {dataset=} or {reward_fn=} should be given")             │
│   120 │                                                                                          │
│   121 │   rich.print("[bold blue]model.learn")                                                   │
│ ❱ 122 │   model.learn()                                                                          │
│   123 │                                                                                          │
│   124 │   return model                                                                           │
│   125                                                                                            │
│                                                                                                  │
│ /home/mila/g/gagnonju/Marg-Li-CoT/our_scratchpad/trlx_repo/trlx/model/accelerate_base_model.py:2 │
│ 37 in learn                                                                                      │
│                                                                                                  │
│   234 │   │   │   │   │   │   self.save()                                                        │
│   235 │   │   │   │   │                                                                          │
│   236 │   │   │   │   │   if self.iter_count % self.config.train.eval_interval == 0:             │
│ ❱ 237 │   │   │   │   │   │   results = self.evaluate()                                          │
│   238 │   │   │   │   │   │                                                                      │
│   239 │   │   │   │   │   │   results.update(stats)                                              │
│   240 │   │   │   │   │   │   results.update(                                                    │
│                                                                                                  │
│ /home/mila/g/gagnonju/Marg-Li-CoT/our_scratchpad/trlx_repo/trlx/model/accelerate_base_model.py:1 │
│ 73 in evaluate                                                                                   │
│                                                                                                  │
│   170 │   │   │                                                                                  │
│   171 │   │   │   # in online setting, compute the reward for validation                         │
│   172 │   │   │   if self.reward_fn:                                                             │
│ ❱ 173 │   │   │   │   rewards = torch.as_tensor(self.reward_fn(samples), dtype=torch.float)      │
│   174 │   │   │   │   mean_reward = rewards.mean()                                               │
│   175 │   │   │   │   columns.append("reward")                                                   │
│   176 │   │   │   │   columns_data.append(rewards)                                               │
│                                                                                                  │
│ /tmp/ipykernel_2172386/3950854667.py:12 in rl_distillation                                       │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_217238

In [6]:
# model_name = "distilgpt2"
# model     = transformers.AutoModelWithLMHead.from_pretrained(model_name)
# tokenizer = transformers.AutoTokenizer      .from_pretrained(model_name)

In [ ]:
# transformers.generation_logits_process.NoRepeatNGramLogitsProcessor
# transformers.generation_logits_process.RepetitionPenaltyLogitsProcessor

In [ ]:
prompts = {
    "cats are ": "and very pretty",
    "dogs are ": "and very cute",
}

prompt_end     = "<|cls|>"
scratchpad_end = "<|cls|>"

for prompt in prompts:
    assert prompt_end not in prompt, prompt

reward_model_model_name = "gpt2"
reward_tokenizer        = transformers.AutoTokenizer           .from_pretrained(reward_model_model_name)
reward_model            = transformers.AutoModelModelWithLMHead.from_pretrained(reward_model_model_name).cuda()



def scratchpad_reward_fn(samples):
    # The idea is to:
    # 1. Extract the associated answers & tokenize the answers
    # 2. Create a mask for the answers
    # 3. Tokenize the samples
    # 4. Concate the samples & answers
    # 5. Run the reward model on the concatenated samples & answers
    # 6. Extract the logp for the answers
    # 7. Return the logp for the answers

    reward_model_inputs  = []
    reward_model_answers = []

    for sample in samples:
        # 1.a Extract the associated answers 
        splitted = sample.split(prompt_end, 1)
        assert len(splitted) == 2, len(splitted)
        question, scratchpad = splitted
        answer = prompts[question]
        reward_model_inputs.append(sample + scratchpad_end)
    
    # 1.b Tokenize the answers
    tokenized_answers = reward_tokenizer(reward_model_answers)
    tokenized_answers_masks = [[1] * len(x) for x in tokenized_answers["input_ids"]]
    tokenizer_samples = reward_tokenizer(reward_model_inputs)
    full_seq = [sample + answer for sample, answer in zip(tokenizer_samples["input_ids"], tokenized_answers["input_ids"])]
    full_seq = reward_tokenizer.pad(full_seq, return_tensors="pt", padding=True)
    full_seq = {k: v.cuda() for k, v in full_seq.items()}

    # MASKS VALUE INPUTS
    tokenized_answers_masks = [[1] * len(x) for x in tokenizer_samples["input_ids"]]
    full_seq_input_masks = [sample_mask + answer_mask for sample_mask, answer_mask in zip(tokenizer_samples["attention_mask"], tokenized_answers_masks)]
    full_seq_input_masks = torch.nn.utils.rnn.pad_sequence(full_seq_input_masks, batch_first=True, padding_value=0)

    reward_model_outputs = reward_model(full_seq["input_ids"], attention_mask=full_seq_input_masks).logsoftmax()

    # MASKS VALUE LOGITS
    tokenized_answers_masks = [[0] * len(x) for x in tokenizer_samples["input_ids"]]
    full_seq_output_masks = [sample_mask + answer_mask for sample_mask, answer_mask in zip(tokenizer_samples["attention_mask"], tokenized_answers_masks)]
    full_seq_output_masks = torch.nn.utils.rnn.pad_sequence(full_seq_output_masks, batch_first=True, padding_value=0)

    # 6. Extract the logp for the entry
    logp = reward_model_outputs.gather(dim=-1, index=full_seq["input_ids"]).squeeze(-1)

    # 7. Return the logp for the answer
    logp *= full_seq_output_masks
    logp = logp.sum(-1)
    assert logp.shape == (len(samples),), logp.shape

    output = logp.mean()
    assert output.shape == (), output.shape
    return output
